In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, precision_score
from sklearn.model_selection import train_test_split

from pathlib import Path
import json

from keras.models import Sequential
from keras.layers import (
    LSTM,
    Bidirectional,
    Conv1D,
    Dense,
    Flatten,
    Input,
    MaxPooling1D,
    TimeDistributed,
    Dropout
)
from keras.initializers import GlorotUniform
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

#### Split Training

In [ ]:
folder = ""

In [ ]:
with open(Path(folder)/'processed_labeled_image_data.npy', 'rb') as f:
    x_data = np.load(f)

with open(Path(folder)/'processed_labeled_image_result.npy', 'rb') as f:
    y_data = np.load(f)

In [ ]:
y_data_categorical = to_categorical(y_data,num_classes=np.unique(y_data).shape[0])

x_train, x_val, y_train, y_val = train_test_split(x_data,y_data_categorical,
                                                    test_size=0.3,
                                                    random_state=42,
                                                    stratify=y_data_categorical)

x_val, _, y_val, _ = train_test_split(x_val,y_val,
                                        test_size=0.34,
                                        random_state=42,
                                        stratify=y_val)

#### CNN LSTM

In [ ]:
model_name = "cnn_lstm"

In [ ]:
model_cnn_lstm = Sequential([
    Input((None, 44, 4)),
    TimeDistributed(
        Sequential([
            Conv1D(filters=32, kernel_size=3, activation="relu"),
            MaxPooling1D(pool_size=2),
            Flatten(),
        ]),
    ),
    LSTM(50, kernel_initializer=GlorotUniform(), activation='relu', return_sequences=True),
    Dropout(0.5),
    LSTM(50, kernel_initializer=GlorotUniform(), activation='relu'),
    Dropout(0.5),
    Dense(6, activation='softmax'),
])
model_cnn_lstm.compile(optimizer="adam", loss='categorical_crossentropy', metrics=["categorical_accuracy","accuracy"])
model_cnn_lstm.summary()

In [ ]:
checkpoint_callback = ModelCheckpoint(
    filepath=folder/f'best_{model_name}_model.keras',       # Filepath to save the model
    monitor='val_accuracy',            # Monitor validation accuracy
    save_best_only=True,               # Save only the best model
    save_weights_only=False,           # Save the full model (architecture + weights)
    mode='max',                        # Save when the monitored quantity is maximized
    verbose=1                          # Verbosity mode
)

In [ ]:
history = model_cnn_lstm.fit(
    x_train,
    y_train,
    validation_data=(x_val,y_val),
    epochs=100,
    batch_size=32,
    callbacks=[checkpoint_callback],
    shuffle=True
)

In [ ]:
with open(Path(folder)/f'training_history_{model_name}.json', 'w') as f:
    json.dump(history.history, f)

In [ ]:
y_pred = model_cnn_lstm.predict(x_train)

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
y_train_classes = np.argmax(y_train, axis=1)

# Calculate precision and recall for multi-class classification
precision = precision_score(y_train_classes, y_pred_classes, average='macro')
recall = recall_score(y_train_classes, y_pred_classes, average='macro')

print(f"Precision: {precision}")
print(f"Recall: {recall}")

In [ ]:
y_pred = model_cnn_lstm.predict(x_val)

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
y_test_classes = np.argmax(y_val, axis=1)

# Calculate precision and recall for multi-class classification
precision = precision_score(y_test_classes, y_pred_classes, average='macro')
recall = recall_score(y_test_classes, y_pred_classes, average='macro')

print(f"Precision: {precision}")
print(f"Recall: {recall}")

#### Bidirectional Model

In [ ]:
model_name = "cnn_bidirectional"

In [ ]:
model_cnn_bi = Sequential([
    Input((None, 44, 4)),
    TimeDistributed(
        Sequential([
            Conv1D(filters=32, kernel_size=3, activation="relu"),
            MaxPooling1D(pool_size=2),
            Flatten(),
        ]),
    ),
    Bidirectional(LSTM(100, kernel_initializer=GlorotUniform(), activation='relu')),
    Dropout(0.5),
    Dense(6, activation='softmax'),
])
model_cnn_bi.compile(optimizer="adam", loss='categorical_crossentropy', metrics=["categorical_accuracy","accuracy"])
model_cnn_bi.summary()

In [ ]:
checkpoint_callback = ModelCheckpoint(
    filepath=f'best_{model_name}_model.keras',       # Filepath to save the model
    monitor='val_accuracy',            # Monitor validation accuracy
    save_best_only=True,               # Save only the best model
    save_weights_only=False,           # Save the full model (architecture + weights)
    mode='max',                        # Save when the monitored quantity is maximized
    verbose=1                          # Verbosity mode
)

In [ ]:
history2 = model_cnn_bi.fit(
    x_train,
    y_train,
    validation_data=(x_val,y_val),
    epochs=100,
    batch_size=32,
    callbacks=[checkpoint_callback],
    shuffle=True
)

In [ ]:
with open(Path(folder)/f'training_history_{model_name}.json', 'w') as f:
    json.dump(history2.history, f)

In [ ]:
y_pred = model_cnn_bi.predict(x_train)

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
y_train_classes = np.argmax(y_train, axis=1)

# Calculate precision and recall for multi-class classification
precision = precision_score(y_train_classes, y_pred_classes, average='macro')
recall = recall_score(y_train_classes, y_pred_classes, average='macro')

print(f"Precision: {precision}")
print(f"Recall: {recall}")

In [ ]:
y_pred = model_cnn_bi.predict(x_val)

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
y_val_classes = np.argmax(y_val, axis=1)

# Calculate precision and recall for multi-class classification
precision = precision_score(y_val_classes, y_pred_classes, average='macro')
recall = recall_score(y_val_classes, y_pred_classes, average='macro')

print(f"Precision: {precision}")
print(f"Recall: {recall}")